# Reinforcement learning from (human) feedback

В этом семинаре я постараюсь поверхностно рассказать про обучение с подкреплением (Reinforcement learning) с обратной связью от человека (human feedback). Поверхносто это будет из-за того, что это новый метод, который к тому же не до конца публичный. RLHF был разработан OpenAI и именно он позволил достичь сильного прироста в качестве генеративных моделей, которое можно наблюдать в ChatGPT. OpenAI не раскрывают всех деталей того, как они этого добились. Есть только обобщенные описание того, как это все работает и большая часть научного/инженерного сообщества в NLP сейчас пытается возпроизвести RLHF. 

В семинаре мы попробуем использовать библиотеку TRL - https://github.com/lvwerra/trl 
Это библиотека связана с Huggingface и поэтому ей удобно пользоваться. (Еще есть похожая библиотека - https://github.com/CarperAI/trlx, я не до конца понимаю насколько она связана с trl, кажется что обе развиваются) 

In [2]:
# %pip install transformers trl wandb

### Reinforcement learning 
Прежде чем разбирать RLHF давайте коротко разберем, что такое Reinforcement learning в целом. RL - это отдельная парадигма в машинном обучении наряду с supervised и unsupervised learning. RL похоже на supervised learning, но модель в нем обучается не на конкретных примерах, а используя положительную/отрицательную обратную связь на свои действия от среды. 
Например, RL применяется в играх (от шахмат до доты) и в робототехнике. В принципе можно обучить supervised модель на действиях людей в игре или в управлении роботом, но тогда получается, что модель будет учиться имитировать людей, а не искать оптимальное решение. В этих задачах можно обойтись и без людей, так как можно автоматически оценить успешность/не успешность - в игре это очки или победа, в роботике это может быть, например, перемещение предмета из точки а в точку б. Обобщоно RL можно представить вот так:

![](https://www.guru99.com/images/1/082319_0514_Reinforceme1.png)

Модель учиться предсказывать действия в конкретной ситуации так, чтобы в итоге прийти к успеху (в игре действия могут быть движение вниз или вверх, а конкретной ситуацией может быть текущий экран). Изначально модель совершает рандомные действия, но постепенно у нее накапливаются последовательности действий, которые ведут к успеху и она начинает их повторять (и избегать тех, которые ведут к поражению).
На практике это может быть очень сложно, так как часто в играх количество потенциально возможных последовательностей очень большое и модель просто не может случайно найти правильные последовательности,и требуется использовать какие-то трюки, специфичные для задачи (например, можно обучать модели играть друг против друга, чтобы можно было играть много игр парралельно в ускоренном режиме и не зависеть от людей). Сейчас RL это по большей части исследовательская область и есть даже мнения, что в итоге RL окажется не нужен (например, так считает Le Cun). Но в некоторых конкретных задачах, RL получилось успешно применить. В том числе, кажется, успешен он и для улучшение языковых моделей.

### Как можно применить RL к языковым моделям?
При генерации текста у нас нет среды, которая могла бы оценить качество текста. Точнее она есть (это человек), но использовать ее напрямую затруднительно, поэтому в RLHF создается отедльная модель, которая будет имитировать человека и оценивать сгенерированные тексты. Такая модель обучается на датасете, который размечается людьми. Сначала с помощью языковой модели генерируется какое-то количество текстов и людей просят выбрать лучшие из них, чтобы сопоставить каждому тексту какое-то число (усредненная оценка). Затем обучается модель, которая предсказывает такие оценки. И на финальном шаге с RL обучаемая генеративная модель генерирует текст и он оценивается с помощью модели обученной на обратной связи от людей. Модель дообучается так, чтобы генерированные тексты получали высокие оценки.

![](https://pbs.twimg.com/media/FfeTNJCUcAEVuo5.jpg:large)

В RLHF в реализации от OpenAI есть еще 1 важный этап - это PPO. Это метод, который используется внутри RL для дообучения модели. Важно про него понимать то, что модель не просто учится максимизировать оценку генерированных текстов, она при этом еще старается минимизировать изменения. В простом fine-tuning частая проблема это то, что дообучаемая модель со временем начинает забывать и теряет в качестве, PPO позволяет этого избежать.

### TRL

Обратная связь от людей в таком фреймворке не обязательна, главное как-то получить модель, которая будет оценивать генерируемые тексты. Можно попытаться использовать доступные модели или датасеты. Давайте попробуем использовать библиотеку trl и дообучим модель OPT-125m генерировать менее негативные тексты использую предобученную модель оценки тональности.

Схематично обучение в TRL устроено вот так:

1) Rollout - это просто генерация текстов, используя исходную языковую модель. Чтобы тексты были релеватными можно взять определенные промты и генерировать их продолжения.

2) Evaluation - это оценивание сгенерированных текстов, в нашем случае мы будет это делать с помощью предобученного классификатора тональности 

3) Optimization - это само обучение. Обратите внимание, что на этом шаге есть active_model и reference_model. Изначально это одна и та же модель, но в процессе обучения мы изменяем active_model так, чтобы она не сильно уходила от изначальное модели (reference_model)

![](https://camo.githubusercontent.com/85d00cf9bca67e33c2d1270b51ff1ac01853b26a8d6bb226b711f859d065b4a6/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f74726c2d696e7465726e616c2d74657374696e672f6578616d706c652d696d616765732f7265736f6c76652f6d61696e2f696d616765732f74726c5f6f766572766965772e706e67)

Код взят из примеров в самой библиотеке и он на torch, но опять же тут ничего сложного (huggingface, токенизация)

In [3]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

[2024-05-30 07:40:45,059] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
 [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible


In [4]:
# тут задаются гиперпараметры
config = PPOConfig(
    model_name="facebook/opt-125m",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

Для генерирования текстов тут используется IMDB review датасет. От каждого текста отрезается случайно какой-то кусочек и модель должна его продолжить.

In [5]:

def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    
    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)
    
    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [6]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/24895 [00:00<?, ? examples/s]

Map:   0%|          | 0/20571 [00:00<?, ? examples/s]

In [46]:
# tokenizer = AutoTokenizer.from_pretrained(config.model_name, padding_side='left')

In [7]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [9]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

Для оценки тональности используется пайплайн из huggingface и предобученная модель

In [10]:
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Для каждого текста она выдает скор негативности и положительности. Для обучения дальше будет использоваться скор положительности

In [11]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.3350486755371094},
  {'label': 'POSITIVE', 'score': -2.726576566696167}]]

In [12]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': -2.2947897911071777},
  {'label': 'POSITIVE', 'score': 2.5570404529571533}]]

Еще одно место, где можно настраивать параметры. Эти параметры мы разбирали на семинаре по GPT они отвечают за генерацию.

In [13]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, 
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

Само обучение. Для небольших моделей оно даже не очень долгое

In [14]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), 
                         total=dataset.num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/160 [00:00<?, ?it/s]

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (12.73) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (16.87) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (10.36) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (10.56) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/t

In [66]:
response_ref, response = [], []
dataset.set_format("pandas")
batch_size = 30
for i in tqdm(range(0, len(dataset), batch_size)):
    batch = dataset[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['query'].tolist(), return_tensors='pt', padding=True)
    output = ref_model.generate(
        input_ids['input_ids'].to(device), max_new_tokens=20, do_sample=False, #**gen_kwargs
    )
    
    response_ref.extend(tokenizer.batch_decode(output, skip_special_tokens=True))
    output = model.generate(
        input_ids['input_ids'].to(device), max_new_tokens=20, do_sample=False, #**gen_kwargs
    )
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))


  0%|          | 0/686 [00:00<?, ?it/s]

In [68]:
#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(dataset["query"], response_ref)]
scores_ref = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(dataset["query"], response)]
scores = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]


/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [71]:
result_df = pd.DataFrame({'query': dataset['query'], 'response (before)': response_ref, 'response (after)': response, 
              'score (before)': scores_ref, 'score (after)': scores}) 

In [73]:
result_df['score_diff'] = result_df['score (after)'] - result_df['score (before)']

In [77]:
result_df.sort_values("score_diff", ascending=False).head(10)

,query,response (before),response (after),score (before),score (after),score_diff
8442,</s>A badly-acted,"A badly-acted, poorly-written, and poorly-written, and poorly-written, and poorly-written,","A badly-acted movie, but I love it. It's a great film. I love it.\nI love",-3.103863,2.823554,5.927417
5367,</s>This movie was a complete,"This movie was a complete waste of time.\nI'm not sure if you're being sarcastic or not, but I think",This movie was a complete masterpiece. I loved it.\nI loved it. It was a great movie.,-3.016589,2.898670,5.915259
953,"</s>This ""film","This ""film"" is a complete waste of time.\nI'm not sure if you're being sarcastic or not","This ""film"" is a great film, and I love it.\nI love it too.",-3.052876,2.849492,5.902368
7756,</s>CQ was the worst film,CQ was the worst film ever made.\nI think it was the worst movie ever made.,CQ was the worst film but I love it. It's so good. It's a great movie.\nI love it,-3.038496,2.845126,5.883622
7637,</s>This so-called prequel,This so-called prequel to the original trilogy is a complete waste of time.\nI'm not sure if you're being,This so-called prequel is a great show. I love it.\nI love it too.,-3.023605,2.849350,5.872955
7604,</s>This film was the worst film,"This film was the worst film I've ever seen.\nI'm not sure if you're being sarcastic or not, but I",This film was the worst film but I love it. It's a great film.\nI love it. It's a great,-3.000785,2.853095,5.853879
1876,</s>The video quality is awful,"The video quality is awful.\nI'm not sure why, but it's not even close to the quality of the video","The video quality is awful, but I love it.\nI love it. It's a great video.",-2.966902,2.864655,5.831557
16610,</s>Spoiler!!! This movie,"Spoiler!!! This movie is so bad.\nI'm not sure if you're being sarcastic or not, but I'm",Spoiler!!! This movie is amazing. I love it.\nI love it.,-2.967174,2.856787,5.823961
17204,</s>The acting in this movie was,The acting in this movie was so bad I couldn't even watch it.\nI'm not sure if it's because I'm,The acting in this movie was amazing. It was a great movie.\nI loved it.,-2.997194,2.823792,5.820986
1749,</s>Quite possibly the worst movie,Quite possibly the worst movie ever made.\nI think it's the worst movie ever made.\nI think it's the,"Quite possibly the worst movie, but I love it.\nI love it too. It's a great movie.",-2.995037,2.813933,5.808970


Видно, что модель теперь генерирует в основном только положительные тексты, даже если промт намекает на негативную тональность.

# DPO

RLHF не единственный подход, который позволяет обучать модели на humam feedback. Самый популярный - Direct Preference Optimization (DPO). Этот метод является в какой-то степени оптимизацией RLHF. Авторы отталкиваются от RLHF c PPO, но с помощью хитрых математических преобразований приходят к лоссу, в котором не нужен RL и не нужна reward модель. В DPO можно обучать модель сразу на фидбек данных (обычно это пары chosen-rejected) как с обычным supervised обучением, но в лоссе в еще сохраняется отсылка к reference модели, чтобы при обучении модель не переобучалась и не забывала то, что она уже знает. 

Лосс для DPO выглядит как-то так: 
![](https://miro.medium.com/v2/resize:fit:1400/0*zE6I3BBUDMN9lfwV.png)

π - тут это модель (θ - та что обучается, а ref - та, что используется как reference), π(y_w | x) - это вероятность, которую модель выдает для лучшего варианта (chosen или winner), π(y_l | x) - вероятность, которая получается для худшего предсказания (rejected или looser). 

Статья про DPO - https://arxiv.org/pdf/2305.18290

Вот тут можно послушать про DPO гораздно подробнее (но при этом понятно) - https://www.youtube.com/watch?v=hvGa5Mba4c8

С DPO также можно использовать общедоступные датасеты с feedback'ом, не нужно их предварительно генерировать модель, которую нужно обучать. Также DPO совместимо с Lora и квантизацией. 

Давайте попробуем обучить mistral на датасете `Intel/orca_dpo_pairs`. Так как это большая модель, то будем обучать только адаптер. 

In [ ]:
# %pip install trl --upgrade

In [ ]:
# %pip install git+https://github.com/huggingface/transformers

In [1]:
# imports
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import DPOTrainer
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training


pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

[2024-05-30 14:03:30,193] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
 [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


In [2]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Модель, которую мы возьмем, уже дообучена на инструкциях и чатах, поэтому промпт для нее нужно приводить в стандартный формат. Huggingface недавно добавил функцию apply_chat_template, которая делается это на основе настроек токенизатора.

In [4]:
def chatml_format(example):
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['question']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    # Format chosen answer
    chosen = example['chosen'] + "<|im_end|>\n"

    # Format rejected answer
    rejected = example['rejected'] + "<|im_end|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

Загрузим датасет и посмотрим на пример

In [5]:
dataset = load_dataset('Intel/orca_dpo_pairs')

Для DPO нам понадобятся `prompt`, `chosen` и `rejected`. Промпт мы соберем из system и question.

In [6]:
(dataset['train'][0])

{'system': '',
 'question': "You will be given a definition of a task first, then some input of the task.\nThis task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.\n\nAFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.\nOutput:",
 'chosen': '[\n  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],\n  ["Ajax Youth Academy", "plays at", "Sportpark De Toekomst"]\n]',
 'rejected': " Sure, I'd be happy to help! Here are the RDF triplets for the input sentence:\n\n[

Возьмем кусочек поменьше для тестирования

In [7]:
dataset_train = dataset['train'].select([i for i in range(10000)])
dataset_test = dataset['train'].select([i for i in range(10000, 10300)])

In [8]:
original_columns = dataset_train.column_names

# отформатируем данные в промпты
dataset_train = dataset_train.map(
    chatml_format,
    remove_columns=original_columns
)
dataset_test = dataset_test.map(
    chatml_format,
    remove_columns=original_columns
)

## Обучение

In [10]:
# загружаем модель в 4bit (то есть у нас будет QLora)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# for name, module in model.named_modules():
#     print(name)

In [12]:
# добавляем адаптер ко всем линейным слоям
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj', 'up_proj']
)

In [13]:
# конфиг для обучения
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir='dpo_train',
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="none",
)

In [14]:
# beta параметр регулирует насколько строго мы штрафуем отклонения от изнчальной модели
# чем больше тем сильнее штрафуем

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset_train,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [15]:
# обучение
dpo_trainer.train()

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.703000
4,0.682100
5,0.659300
6,0.616700
7,0.588200
8,0.557800
9,0.530600
10,0.537900


KeyboardInterrupt: 

In [16]:
# сохранение (только адаптер!)
dpo_trainer.model.save_pretrained('dpo_mistral')

## Тестирование обученной модели

Как и с RLHF давайте сгенерируем продолжения старой и новой моделью и посмотрим, что получается

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
model_ref = AutoModelForCausalLM.from_pretrained(model_name, device_map='cuda', torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
device = model_ref.device
response_ref = []
dataset_test.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model_ref.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=50, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response_ref.extend(tokenizer.batch_decode(output, skip_special_tokens=True))


  0%|          | 0/50 [00:00<?, ?it/s]

In [20]:
model = PeftModel.from_pretrained(model_ref, "dpo_mistral")

In [21]:
response = []
dataset_test.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=50, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))


  0%|          | 0/50 [00:00<?, ?it/s]

In [22]:
result_df = pd.DataFrame({'prompt': dataset_test['prompt'][:100], 'response (before)': response_ref, 'response (after)': response}) 

In [23]:
result_df

,prompt,response (before),response (after)
0,"<|im_start|>system\nYou are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.<|im_end|>\n<|im_start|>user\nU.S. expels Venezuelan diplomat in Miami\n\nBy the CNN Wire Staff\n\nupdated 12:33 PM EST, Sun January 8, 2012\n\nWashington (CNN) -- Venezuela's consul general in Miami has been declared to be persona non grata and must leave the United States, a State Department spokesman said Sunday.\n\nSpokesman William Ostick declined to comment on specific details behind the decision to expel Livia Acosta Noguera, who has headed Venezuela's consulate in Miami since March 2011.\n\nThe Venezuelan Embassy in Washington was informed of the decision Friday, the spokesman said in a written statement, and the State Department said Acosta must depart the United States by Tuesday.\n\nIt was unclear Sunday whether she remained in the United States.\n\nLast month, a group of American lawmakers said they had ""grave concerns"" about Acosta and called for an investigation after the Spanish-language TV channel Univision aired a documentary alleging that she was among a group of Venezuelan and Iranian diplomats who expressed interest in an offer from a group of Mexican hackers to infiltrate the websites of the White House, the FBI, the Pentagon and U.S. nuclear plants.\n\nThe evidence that the plot was real, according to Univision, are secret recordings with diplomats who ask questions about what the hackers can do and promise to send information to their governments.\n\nUnivision interviewed a purported Mexican whistle-blower -- a student at the National Autonomous University of Mexico named Juan Carlos Munoz Ledo. The student told Univision he was recruited by a leftist professor who wanted to wage cyberattacks on the United States and its allies.\n\nMunoz told Univision he secretly recorded a meeting in 2008 with Acosta, who was then the cultural attache of the Venezuelan Embassy in Mexico. According to a recording Univision aired as part of its report, Acosta is heard saying that she can send the information gathered by the hackers straight to Venezuelan President Hugo Chavez.\n\nChavez has called the report ""lies.""\n\nOne of the Iranian diplomats told Univision that although he, indeed, was presented with a hacking plot by the Mexican group, he turned it down, in part because he thought they were CIA agents.\n\nIn a letter to U.S. Secretary of State Hillary Clinton last month, Rep. Ileana Ros-Lehtinen, Rep. David Rivera, Rep. Mario Diaz-Balart and Rep. Albio Sires asked the State Department to require Acosta's ""immediate departure"" from the United States if the Univision report proved true.\n\nLast month a State Department spokesman said the United States did not know about the alleged plot, but that it found the Univision allegations ""very disturbing.""\n\nHowever, ""we don't have any information, at this point, to corroborate it,"" State Department spokesman Mark Toner said.\n\nCNN's Jill Dougherty, Juan Carlos Lopez and Mariano Castillo contributed to this report.\n\nMost popular stories right now\n\nWrite a one or two sentence summary.<|im_end|>\n<|im_start|>assistant\n","A Venezuelan diplomat in Miami has been asked to leave the United States because some people are worried that she might have been involved in a plan to hack important American websites. The Venezuelan government says this is not true, and the diplom","The United States has asked a Venezuelan diplomat in Miami to leave the country because they are worried about her connections to a group of Mexican hackers who wanted to attack important American websites. The Venezuelan diplomat, Livia Acosta"
1,"<|im_start|>system\nYou are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.<|im_end|>\n<|im_start|>user\nWhich UK member of parliament resigned in June 2008 to fight the Haltemprice and Howden election on a civil liberties pl

Разница в этот раз гораздо менее заметна так как фидбек который мы пытаемся учесть в модели гораздо более абстрактный и разнообразный. Но разница есть!